## Refer to [readme.pdf](https://drive.google.com/file/d/1vbNsY0Olfs9IShPD0dxdACSXCzhT1MU9/view?usp=sharing) for detailed instructions to setup the environment before running this file.

## Refer to the anonymised drive folder [Evaluation_All](https://drive.google.com/drive/folders/19BM3f4UeywAuhPi1xk7iJIXP0q5pi2kB?usp=sharing) for the entire setup and directory structure.

In [ ]:
!pip install av

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Enter path to google drive folder
%cd /content/drive/MyDrive/HP_BS_T17/Evaluation_All

In [ ]:
PATH_labelmap = "./label_map_k600.txt"
PATH_model =  './checkpoint/Blackbox_P2.pt'  #Add desired model checkpoint 
PATH_files = './k600_val/'

#Imports
import torch
import glob
import pandas as pd
import os
import re
from torchvision.io import read_video as rv
import numpy as np
from sklearn.metrics import top_k_accuracy_score as tkscore


import warnings
warnings.filterwarnings("ignore")

"""# Load files"""

def class_to_label(class_):
    class_= re.sub(r'[^\w\s]', '', class_)
    class_ = class_.replace('_','')
    class_ = class_.replace(' ','')
    #print(class_)
    df=pd.read_csv(PATH_labelmap,names=['class'])
    df['class']=df['class'].str.replace(' ', '')
    df["class"] = df['class'].str.replace('[^\w\s]','')
    #print(df['class'])
    #print(class_)
    ret_= df.index[df['class']==class_]
    #print(ret_)
    return ret_[0]

def sample_class_paths(PATH, nvids):
    d_vid={}
    root,dirs,files=next(os.walk(PATH))
    for dir in dirs:
        root=str(root)
        dir=str(dir)
        _,_,file=next(os.walk(root+'/'+dir))
        d_vid['{}'.format(dir)]=[file[:nvids]]
    return d_vid

def initialize_model(checkpoint):
  model = torch.load(checkpoint)
  model = model.eval()
  return model

def parse_video(PATH):
    video=rv(PATH)
    samp=video[0]
    #print(samp.size())
    samp1= samp.type(torch.FloatTensor).permute(3,0,1,2)
    #samp1=samp1[:,:,:128,:128] Use in case of OOM errors
    #print(samp1.size())
    return samp1

def eval(num_videos = 8, num_classes = 400, checkpoint = None):
  with torch.no_grad():
    true_labels = []
    pred_probs = []   
    if(checkpoint is None):
      print("Please provide model checkpont")
      return False
    model = initialize_model(checkpoint)
    dict_paths = sample_class_paths(PATH_files, num_videos)
    counter = 0
    for keys in dict_paths.keys():
      for files in dict_paths[keys][0]:
        true_labels.append(class_to_label(keys))
        vid_tensor = parse_video(PATH_files+f'/{keys}/{files}') # Size(channels, frames, height, width)
        ## Error handling code for broken videos
        try:  
          model_logits = model(torch.unsqueeze(vid_tensor[:,:,:,:].cuda(),dim=1))
          #model_logits = model(torch.unsqueeze(vid_tensor[:,:32,:,:].cuda(),dim=0))  #Use in case of OOM error
        except:
          model_logits = torch.zeros(1,num_classes)
        model_probabs = torch.nn.functional.softmax(model_logits,dim=0)
        pred_probs.append(model_probabs)
        counter+=1
        print(f"Video number = {counter}")

    true_label_arr = np.array(true_labels) # (3200,1)
    temp_tensor = torch.zeros(counter,num_classes)
    for i in range(counter):
      temp_tensor[i] = pred_probs[i].cpu()
    pred_probs_arr = np.array(temp_tensor)

    accuracy_top5 = tkscore(true_label_arr,pred_probs_arr,k=5)
    print(f"Top 5 accuracy is {accuracy_top5*100} %")
    return accuracy_top5

In [ ]:
num_classes = 600 #Kinetics 600 dataset
num_videos = 24 #Number of videos from each class to pick for evaluation

with torch.no_grad():
  acc = eval(checkpoint = PATH_model, num_videos = num_videos, num_classes = num_classes)